### Custom MBA courses RAG project

In [1]:
# Go one level up in the directories hierarchy to access src directory and codes
import sys
import os
# Add project root to Python path
project_root = os.path.abspath("..")  # go one level up from notebooks/
sys.path.append(project_root)

In [2]:
# Setup necessary models for routing, chatting and embedding
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from core.config.config import Config
from google.genai import types

router_llm = GoogleGenAI(
    model = Config.ROUTER_LLM,
    api_key = Config.GOOGLE_API_KEY,
    generation_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(thinking_budget = 0),
        temperature = Config.ROUTER_LLM_TEMPERATURE,
    ),
    max_tokens = Config.ROUTER_LLM_MAX_TOKENS
)

chat_llm = GoogleGenAI(
    model = Config.CHAT_LLM,
    api_key = Config.GOOGLE_API_KEY,
    generation_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(thinking_budget = 0),
        temperature = Config.CHAT_LLM_TEMPERATURE,
    ),
    max_tokens = Config.CHAT_LLM_MAX_TOKENS
)

embed_model = HuggingFaceEmbedding(
    model_name = Config.EMBEDDING_MODEL
)

c:\Users\Sanzhar\workspace\github.com\kaydarovv\ai_agent_learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the collections name and description from my custom JSON file
import json

docs_path = "../documents"
collections_mba_json = docs_path + "/collections_mba.json"

with open(collections_mba_json, "r", encoding = "utf-8") as file:
    COLLECTIONS_MBA = json.load(file)

for collections_name, collection_description in COLLECTIONS_MBA.items():
    COLLECTIONS_MBA[collections_name] = (" \n ").join([line.strip() for line in collection_description.splitlines()[1:-2]])

In [4]:
# Let's test our RAG module defined in class to see how well it is refactored
import nest_asyncio
from core.src.rag_workflow import RagSystem
nest_asyncio.apply()

rag_instance = RagSystem(
    router_llm = router_llm,
    chat_llm = chat_llm,
    embed_model = embed_model,
    docs_path = docs_path,
    collections = COLLECTIONS_MBA
)

2025-12-15 10:35:46,126 - WARNING - Ignoring wrong pointing object 9 0 (offset 0)
2025-12-15 10:35:46,127 - WARNING - Ignoring wrong pointing object 20 0 (offset 0)
2025-12-15 10:35:46,128 - WARNING - Ignoring wrong pointing object 22 0 (offset 0)
2025-12-15 10:35:46,128 - WARNING - Ignoring wrong pointing object 30 0 (offset 0)
2025-12-15 10:35:46,129 - WARNING - Ignoring wrong pointing object 36 0 (offset 0)
2025-12-15 10:35:46,130 - WARNING - Ignoring wrong pointing object 49 0 (offset 0)
2025-12-15 10:35:46,130 - WARNING - Ignoring wrong pointing object 55 0 (offset 0)
2025-12-15 10:35:46,131 - WARNING - Ignoring wrong pointing object 82 0 (offset 0)
2025-12-15 10:35:46,131 - WARNING - Ignoring wrong pointing object 84 0 (offset 0)
2025-12-15 10:35:46,132 - WARNING - Ignoring wrong pointing object 95 0 (offset 0)
2025-12-15 10:35:46,133 - WARNING - Ignoring wrong pointing object 112 0 (offset 0)
2025-12-15 10:35:46,133 - WARNING - Ignoring wrong pointing object 115 0 (offset 0)
202

In [5]:
# Let's test the query and memory of the RAG system
user_name = "TestUser"
user_id = "test_user_id"
user_query = "Are there any rules when it comes to Financial Accounting?"

rag_response = await rag_instance.query(user_name, user_id, user_query)
print(rag_response)

2025-12-15 10:37:49,293 - INFO - AFC is enabled with max remote calls: 10.
2025-12-15 10:37:50,430 - INFO - Selecting retriever 0: This choice explicitly mentions 'generally accepted accounting principles in the US (US GAAP) and around the world (IFRS)', which are the rules governing financial accounting..
2025-12-15 10:37:50,455 - INFO - AFC is enabled with max remote calls: 10.
2025-12-15 10:37:51 | INFO     | core.src.rag_workflow:_get_or_create_chat_engine:158 - Creating a new chat engine for user {user_id}
2025-12-15 10:37:51,148 - INFO - AFC is enabled with max remote calls: 10.


TestUser, yes, there are rules when it comes to Financial Accounting. Financial accounting produces reports that are prepared under rules to inform external parties. Specifically, Generally Accepted Accounting Principles (GAAP), such as US-GAAP and Indian-GAAP, are part of the information system for financial accounting. External rules like IFRS and US-GAAP are also mentioned in relation to financial accounting.
